### Interpreting Estimated Coefficients

 **1. Suppose that we would like to know how much families in the US are spending on recreation annually. We've estimated the following model:**

𝑒𝑥𝑝𝑒𝑛𝑑𝑖𝑡𝑢𝑟𝑒=873+0.0012𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒+0.00002𝑎𝑛𝑛𝑢𝑎𝑙_𝑖𝑛𝑐𝑜𝑚𝑒2−223.57ℎ𝑎𝑣𝑒_𝑘𝑖𝑑𝑠

Our main purpose of this model is predicting US families' annual recreation spending.And the above equation has done the job for us.Lets interpret what that chunk of equation means in real term.Our constant or bias term for our model is 873.It is the value that our expenditure prediction value assumes when when there is zero annual income.the second part in the equation can be interpreted as given that the effects of all other variables is zero for every $100 increase in annual income there is an increase of expenditure by 12 cents i.e 100 X 0.0012 and additionally as in the third part of the equation, There is an increase of expenditure by 20 cents i.e 100 times 100 times 0.00002 which totals up to 32 cents of average increase in expenditure for every 100 average increase in annual income.The fourth part of the equation can be interpreted as families with kids spend on average 223.57 less than families without kids on recreation.This is possibly because they have another priorities to spend on.<br/>
For our estimation to be significant,we need to make inference test.the two important statistic for our inference purpose are the t-statstic and the p-value associated with it.If the p-value for each and every coefficient is < 0.1 then our feature has a significant effect on our outcome variable otherwise the feature's effect on our outcome variable is negligible or about zero.


**2.**

In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'weatherinszeged'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

weather_df = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
engine.dispose()


In [13]:
weather_df.head()

,date,summary,preciptype,temperature,apparenttemperature,humidity,windspeed,windbearing,visibility,loudcover,pressure,dailysummary
0,2006-03-31 22:00:00+00:00,Partly Cloudy,rain,9.472,7.389,0.890,14.120,251.000,15.826,0.000,1015.130,Partly cloudy throughout the day.
1,2006-03-31 23:00:00+00:00,Partly Cloudy,rain,9.356,7.228,0.860,14.265,259.000,15.826,0.000,1015.630,Partly cloudy throughout the day.
2,2006-04-01 00:00:00+00:00,Mostly Cloudy,rain,9.378,9.378,0.890,3.928,204.000,14.957,0.000,1015.940,Partly cloudy throughout the day.
3,2006-04-01 01:00:00+00:00,Partly Cloudy,rain,8.289,5.944,0.830,14.104,269.000,15.826,0.000,1016.410,Partly cloudy throughout the day.
4,2006-04-01 02:00:00+00:00,Mostly Cloudy,rain,8.756,6.978,0.830,11.045,259.000,15.826,0.000,1016.510,Partly cloudy throughout the day.


In [14]:
weather_df['target_var'] = weather_df['apparenttemperature'] - weather_df['temperature']
Y = weather_df['target_var']
X = weather_df[['humidity', 'windspeed']]

In [6]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             target_var   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Sat, 14 Sep 2019   Prob (F-statistic):               0.00
Time:                        17:54:59   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4381      0.021    115.948      0.000       2.397       2.479
humidity      -3.0292      0.024   -126.479      0.000      -3.076      -2.982
windspeed     -0.1193      0.001   -176.164      0.000      -0.121      -0.118
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.267
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                          -0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

From the p-values in our result table we can infere that all the coefficients have significant effect on the model.The signs of the two explaratory variables is negative which is consistent with the signs from our previous analysis.Interpreting the coefficients of the equation with respect to the two independent variables and the difference of the two temperature variables as dependent variable;the bias term is 2.44 for every increase on the humidity by 1 there is a decrease in the difference values in temperature by 3.03 given the effect of other variables is zero.And for every increase on the windspeed by 1 there is a decrease in the difference values in temperature by 0.12 given the effect of other variables is zero.

* Now lets create an interaction variable between humidity and windspeed and add it to the exploratory data.

In [8]:
weather_df['humidity_windspeed'] = weather_df['humidity'] * weather_df['windspeed']

In [10]:
X = weather_df[['humidity', 'windspeed', 'humidity_windspeed']]
Y = Y = weather_df['target_var']

In [11]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             target_var   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Sat, 14 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:18:58   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0839      0.033      2.511      0.012       0.018       0.149
humidity               0.1775      0.043      4.133      0.000       0.093       0.262
windspeed              0.0905      0.002     36.797      0.000       0.086       0.095
humidity_windspeed    -0.2971      0.003    -88.470      0.000      -0.304      -0.291
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.265
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                          -0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

* The coefficients are statistically significant because all the p-values of the coefficients from the result table in the above are greater than the significance level.

* The signs of the coefficients of the humidity and windspeed were changed from negative to positive.

* Interpreting the new coefficients;our bias value this time is 0.08,it is a constant value. The next coefficient indicates that for every increase in humidity by 1, the difference in temperature increases by 0.18 if treated alone with out the effect of windspeed, and if windspeed is treated with out the effect of humidity,for every increase in 1 in windspeed, the target variable increases by 0.1.Howevery the combined effect of humidity and windspeed decreases the difference of the temperature by 0.3 for every unit increase in their combination.

**3.**

In [15]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host
                                                            ,postgres_port,postgres_db))
house_prices = pd.read_sql_query('select * from houseprices', con = engine)
engine.dispose()

In [16]:
house_prices.head()

,id,mssubclass,mszoning,lotfrontage,lotarea,street,alley,lotshape,landcontour,utilities,...,poolarea,poolqc,fence,miscfeature,miscval,mosold,yrsold,saletype,salecondition,saleprice
0,1,60,RL,65.000,8450,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,None,Reg,Lvl,AllPub,...,0,None,None,None,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,None,IR1,Lvl,AllPub,...,0,None,None,None,0,12,2008,WD,Normal,250000


In [17]:
house_prices['is_z_centralair'] = pd.get_dummies(house_prices.centralair, drop_first=True)
num_columns = ['overallqual','totalbsmtsf', 'firstflrsf', 'grlivarea','garagecars', 'garagearea']
num_columns.append('is_z_centralair')
X = house_prices[num_columns]
Y = house_prices['saleprice']

In [18]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     668.4
Date:                Sat, 14 Sep 2019   Prob (F-statistic):               0.00
Time:                        18:48:18   Log-Likelihood:                -17493.
No. Observations:                1460   AIC:                         3.500e+04
Df Residuals:                    1452   BIC:                         3.504e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.106e+05   5685.210    -19.456      0.000   -1.22e+05   -9.95e+04
overallqual      2.351e+04   1095.574     21.456      0.000    2.14e+04    2.57e+04
totalbsmtsf        23.5468      4.319      5.452      0.000      15.075      32.019
firstflrsf         11.5981      5.023      2.309      0.021       1.745      21.451
grlivarea          43.8685      2.687     16.326      0.000      38.598      49.139
garagecars       1.429e+04   3012.969      4.742      0.000    8377.231    2.02e+04
garagearea         14.5942     10.459      1.395      0.163      -5.922      35.110
is_z_centralair  1.192e+04   4334.626      2.749      0.006    3414.876    2.04e+04
==============================================================================
Omnibus:                      424.543   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            39500.235
Skew:                          -0.253   Prob(JB):                         0.00
Kurtosis:                      28.477   Cond. No.                     1.46e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.46e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

* Except the coefficient of the 'garagearea' which its p-value is over 0.1, all the other coefficients are statistically significant.

Lets remove the 'garagearea' and re-run and check our model again.

In [20]:
num_columns = ['overallqual','totalbsmtsf', 'firstflrsf', 'grlivarea','garagecars']
num_columns.append('is_z_centralair')
X = house_prices[num_columns]
Y = house_prices['saleprice']

In [21]:
X = sm.add_constant(X)
results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     778.9
Date:                Sun, 15 Sep 2019   Prob (F-statistic):               0.00
Time:                        16:03:25   Log-Likelihood:                -17494.
No. Observations:                1460   AIC:                         3.500e+04
Df Residuals:                    1453   BIC:                         3.504e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const           -1.111e+05   5678.251    -19.557      0.000   -1.22e+05   -9.99e+04
overallqual      2.346e+04   1095.388     21.416      0.000    2.13e+04    2.56e+04
totalbsmtsf        24.1132      4.301      5.606      0.000      15.676      32.550
firstflrsf         12.0477      5.014      2.403      0.016       2.212      21.884
grlivarea          44.0467      2.685     16.406      0.000      38.780      49.313
garagecars        1.77e+04   1757.438     10.073      0.000    1.43e+04    2.12e+04
is_z_centralair  1.214e+04   4333.037      2.802      0.005    3643.014    2.06e+04
==============================================================================
Omnibus:                      412.217   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            36304.945
Skew:                          -0.196   Prob(JB):                         0.00
Kurtosis:                      27.426   Cond. No.                     1.42e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.42e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

There was a very little difference in the coefficients but overall there was no noticeable impact.

* So now lets Interpret the statistically significant coefficients by quantifying their relations with the house prices and then find out which features have a more prominent effect on house prices.

if the effect of all the features on the housing price('salesprice') is zero,the value of our dependent variable(target variable) is -111100.it is the bias constant of our equation.For every unit increase in 'overallqual'(over all quality rating) there is an increase in the cost of housing by $23460.And for every unit increase in the 'totalbsmtsf' there is an increase on the cost of housing by 24.11.For every unit increase in 'firstflrsf' there is an increase of 12.05 in the cost of the housing.for every unit increase in 'grlivarea' there is an increase on the cost of housing by 44.05 and so forth for the contiunuous variables.The only different interpretation is for the categorical variable 'centralair';in this case the interpretation is quite different.a house with 'centralair' as yes worth 12,140 more than the house without. 

The Features with the most prominent effect on our outcome variable 'house_prices' are the 'overallqual','garagecars' and the 'is_z_centralair' with a coefficient values of 23460,17700 and 12140 respectively.

* The results seems reasonable to me,but to confirm that, checking if the Gauss-Markov conditions are met or not is a good strategy.